In [ ]:
# import lib
import cv2
import numpy as np
import mediapipe as mp
from collections import deque

# different arrays to handel colour points
bpoints = [deque(maxlen=1024)]
gpoints = [deque(maxlen=1024)]
rpoints = [deque(maxlen=1024)]
ypoints = [deque(maxlen=1024)]

#indexe used to mark the points in particular arrays of sepecific colour
blue_index = 0
green_index = 0
red_index = 0
yellow_index = 0

#the  kernel to be used for dilation purpose
kernel = np.ones((5,5),np.uint8)

colors = [(255,0,0), (0,255,0), (0,0,255),(0,255,255)]
colorIndex = 0

#white canvas setup
paintWindow = np.zeros((471,636,3)) + 255
paintWindow = cv2.rectangle(paintWindow, (40,1),(140,65),(0,0,0),2)
paintWindow = cv2.rectangle(paintWindow, (160,1),(255,65),(255,0,0),2)
paintWindow = cv2.rectangle(paintWindow, (275,1),(370,65),(0,255,0),2)
paintWindow = cv2.rectangle(paintWindow, (390,1),(485,65),(0,0,255),2)
paintWindow = cv2.rectangle(paintWindow, (505,1),(600,65),(0,255,255),2)

cv2.putText(paintWindow, "CLEAR", (49,33),cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0,0,0),2, cv2.LINE_AA)
cv2.putText(paintWindow, "BLUE", (185,33),cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0,0,0),2, cv2.LINE_AA)
cv2.putText(paintWindow, "GREEN", (298,33),cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0,0,0),2, cv2.LINE_AA)
cv2.putText(paintWindow, "RED", (420,33),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0),2, cv2.LINE_AA)
cv2.putText(paintWindow, "YELLOW", (520,33),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0),2, cv2.LINE_AA)
cv2.namedWindow("Paint", cv2.WINDOW_AUTOSIZE)

# instalize mediapipe
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=2,min_detection_confidence=0.7)
#this is used to draw land marks on detected hand
mpDraw = mp.solutions.drawing_utils

#initialize camera
cap = cv2.VideoCapture(0)
ret, frame_temp = cap.read()
while ret:
    ret, frame = cap.read()

    x,y,c=frame.shape

    #flip the frame
    frame = cv2.flip(frame,1)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    frame = cv2.rectangle(frame,(40,1),(140,65),(0,0,0),2)
    frame = cv2.rectangle(frame,(160,1),(255,65),(255,0,0),2)
    frame = cv2.rectangle(frame,(275,1),(370,65),(0,255,0),2)
    frame = cv2.rectangle(frame,(390,1),(485,65),(0,0,255),2)
    frame = cv2.rectangle(frame,(505,1),(600,65),(0,255,255),2)
    cv2.putText(frame, "CLEAR", (49, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2,cv2.LINE_AA)
    cv2.putText(frame, "BLUE", (185, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2,cv2.LINE_AA)
    cv2.putText(frame, "GREEN", (298, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2,cv2.LINE_AA)
    cv2.putText(frame, "RED", (420, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "YELLOW", (520, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2,cv2.LINE_AA)
    #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    #get hand landmark predication
    result = hands.process(framergb)

    #post process the result
    if result.multi_hand_landmarks:
        landmarks = []
        for handslms in result.multi_hand_landmarks:
            for lm in handslms.landmark:
                ## print(id,lm)
                # print (lm.x)
                # print(lm.y)
                #Adjust according to your frame size
                lmx = int(lm.x*640)
                lmy = int(lm.y*640)

                landmarks.append([lmx, lmy])
            # Drawing landmarks on frames
            mpDraw.draw_landmarks(frame, handslms, mpHands.HAND_CONNECTIONS)
        fore_finger = (landmarks[8][0], landmarks[8][1])
        center = fore_finger
        thumb = (landmarks[4][0],landmarks[4][1])
        cv2.circle(frame, center, 3, (0,255,0),-1)
        print(center[1]-thumb[1])
        if (thumb[1]-center[1]<30):
            bpoints.append(deque(maxlen=512))
            blue_index += 1
            gpoints.append(deque(maxlen=512))
            green_index += 1
            rpoints.append(deque(maxlen=512))
            red_index += 1
            ypoints.append(deque(maxlen=512))
            yellow_index += 1
        elif center[1]<=65:
            if 40<=center[0]<=140: #clear Button
                bpoints = [deque(maxlen=512)]
                gpoints = [deque(maxlen=512)]
                rpoints = [deque(maxlen=512)]
                ypoints = [deque(maxlen=512)]

                blue_index = 0
                green_index = 0
                red_index = 0
                yellow_index = 0

                paintWindow[67:,:,:] = 255
            elif 160 <= center[0]<=255:
                colorIndex = 0 #blue
            elif 275 <= center[0]<=370:
                colorIndex = 1 # green
            elif 390 <= center[0]<=485:
                colorIndex = 2 #red
            elif 505 <= center[0]<= 600:
                colorIndex = 3 # yellow
        else:
            if colorIndex == 0:
                bpoints[blue_index].appendleft(center)
            elif colorIndex == 1:
                gpoints[green_index].appendleft(center)
            elif colorIndex == 2:
                rpoints[red_index].appendleft(center)
            elif colorIndex == 3:
                ypoints[yellow_index].appendleft(center)
    #append the next deques when nothing is detected to avois messing up
    else:
        bpoints.append(deque(maxlen=512))
        blue_index += 1
        gpoints.append(deque(maxlen=512))
        green_index += 1
        rpoints.append(deque(maxlen=512))
        red_index += 1
        ypoints.append(deque(maxlen=512))
        yellow_index += 1

    # draw lines of all the colors
    points = [bpoints,gpoints,rpoints,ypoints]

    for i in range(len(points)):
        for j in range(len(points[i])):
            for k in range(1, len(points[i][j])):
                if points[i][j][k-1] is None or points[i][j][k] is None:
                    continue
                cv2.line(frame,points[i][j][k-1],points[i][j][k], colors[i],2)
                cv2.line(paintWindow, points[i][j][k-1], points[i][j][k],colors[i],2)
    cv2.imshow("Output",frame)
    cv2.imshow("paint",paintWindow)

    if cv2.waitKey(1) == ord('q'):
        break
# release the web cam and destory all active window
cap.release()
cv2.destroyAllWindows()

C:\Users\HP\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


-138
-149
-156
-154
-155
-110
-193
-203
-205
-204
-204
-204
-204
-203
-205
-204
-204
-203
-204
-203
-202
-199
-198
-194
-187
-180
-175
-176
-171
-172
-167
-167
-163
-162
-162
-157
-155
-150
-151
-148
-147
-144
-145
-142
-138
-136
-135
-131
-127
-123
-120
-116
-108
-86
-88
-75
-70
-60
-44
-42
-36
-34
-38
-38
-45
-46
-55
-56
-59
-56
-55
-30
1
5
-12
-17
-18
-14
-14
-19
-17
-18
-19
-14
-19
-16
-16
-10
-12
-13
-12
-15
-12
-12
-11
-19
-22
-22
-20
-19
-29
-17
16
5
-16
-14
-20
-5
-16
-15
-20
-15
-15
-29
-47
-58
-76
-91
-91
-112
-115
-123
-135
-139
-144
-150
-149
-154
-157
-158
-157
-159
-158
-160
-162
-162
-165
-167
-164
-161
-158
-152
-149
-145
-149
-151
-154
-160
-165
-169
-171
-176
-173
-178
-170
-153
-151
-106
-121
-243
-243
-229
-231
-231
-239
-236
-234
-234
-238
-235
-235
-244
-229
-230
-231
-229
-227
-229
-229
-221
-217
-216
-209
-207
-212
-114
-104
-120
-119
-115
-116
-116
-116
-115
-111
-109
-92
-95
-89
-83
-81
-80
-79
-77
-77
-76
-74
-75
-78
-77
-79
-77
-78
-80
-81
-79
-79
-83
-86
-8

In [2]:
pip install opencv-python

   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB 1.3 MB/s eta 0:00:31
   ---------------------------------------- 0.1/38.8 MB 1.6 MB/s eta 0:00:24
   ---------------------------------------- 0.3/38.8 MB 2.0 MB/s eta 0:00:19
   ---------------------------------------- 0.4/38.8 MB 2.3 MB/s eta 0:00:17
    --------------------------------------- 0.8/38.8 MB 3.9 MB/s eta 0:00:10
   - -------------------------------------- 1.4/38.8 MB 5.5 MB/s eta 0:00:07
   -- ------------------------------------- 2.4/38.8 MB 7.9 MB/s eta 0:00:05
   --- ------------------------------------ 3.0/38.8 MB 8.6 MB/s eta 0:00:05
   --- ------------------------------------ 3.0/38.8 MB 8.6 MB/s eta 0:00:05
   ---- ----------------------------------- 3.9/38.8 MB 9.2 MB/s eta 0:00:04
   ---- ----------------------------------- 4.8/38.8 MB 10.2 MB/s eta 0:00:04
   ----- ---------------------------------- 5.0/38.8 MB 10.3 MB/s eta 0:00:04
   -

In [1]:
pip install opencv-python-headless


   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB 495.5 kB/s eta 0:01:19
   ---------------------------------------- 0.1/38.8 MB 751.6 kB/s eta 0:00:52
   ---------------------------------------- 0.1/38.8 MB 751.6 kB/s eta 0:00:52
   ---------------------------------------- 0.2/38.8 MB 1.1 MB/s eta 0:00:34
   ---------------------------------------- 0.2/38.8 MB 1.1 MB/s eta 0:00:34
    --------------------------------------- 0.5/38.8 MB 1.7 MB/s eta 0:00:23
    --------------------------------------- 0.7/38.8 MB 2.1 MB/s eta 0:00:19
    --------------------------------------- 1.0/38.8 MB 2.3 MB/s eta 0:00:17
   - -------------------------------------- 1.0/38.8 MB 2.1 MB/s eta 0:00:18
   - -------------------------------------- 1.5/38.8 MB 2.9 MB/s eta 0:00:13
   - ---------

In [2]:
pip install mediapipe

   ---------------------------------------- 0.0/50.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/50.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/50.8 MB 787.7 kB/s eta 0:01:05
   ---------------------------------------- 0.1/50.8 MB 930.9 kB/s eta 0:00:55
   ---------------------------------------- 0.1/50.8 MB 930.9 kB/s eta 0:00:55
   ---------------------------------------- 0.2/50.8 MB 958.4 kB/s eta 0:00:53
   ---------------------------------------- 0.2/50.8 MB 958.4 kB/s eta 0:00:53
   ---------------------------------------- 0.3/50.8 MB 930.9 kB/s eta 0:00:55
   ---------------------------------------- 0.3/50.8 MB 930.9 kB/s eta 0:00:55
   ---------------------------------------- 0.4/50.8 MB 897.8 kB/s eta 0:00:57
   ---------------------------------------- 0.4/50.8 MB 897.8 kB/s eta 0:00:57
   ---------------------------------------- 0.5/50.8 MB 887.1 kB/s eta 0:00:57
   ---------------------------------------- 0.5/50.8 MB 925.6 kB/s eta

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\HP\\anaconda3\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



In [3]:
pip install mediapipe --user

  Using cached mediapipe-0.10.14-cp312-cp312-win_amd64.whl.metadata (9.9 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached jax-0.4.30-py3-none-any.whl.metadata (22 kB)
  Using cached jaxlib-0.4.30-cp312-cp312-win_amd64.whl.metadata (1.1 kB)
  Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached sounddevice-0.4.7-py3-none-win_amd64.whl.metadata (1.4 kB)
  Using cached ml_dtypes-0.4.0-cp312-cp312-win_amd64.whl.metadata (20 kB)
Using cached mediapipe-0.10.14-cp312-cp312-win_amd64.whl (50.8 MB)
Using cached sounddevice-0.4.7-py3-none-win_amd64.whl (200 kB)
Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
Using cached jax-0.4.30-py3-none-any.whl (2.0 MB)
Using cached jaxlib-0.4.30-cp312-cp312-win_amd64.whl (52.0 MB)
Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-win_amd64.whl (45.5 MB)
Using cached ml_dtypes-0.4.0-cp312-cp312-win_amd64.whl (127 kB)
Note: you may need to restart the kernel to use upd